In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

import os
import re
import sys
import glob
import time
import tarfile
from six.moves import urllib
import pickle

import argparse
import tempfile
import pandas as pd

import scipy

from random import shuffle

from sklearn import ensemble as ens
from sklearn import metrics as met
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix

import eto_utils as utils

# import xgboost as xgb
import lightgbm as lgb 

In [2]:
def context_feat(pred,dog_id_column_list,c_left,c_right):
    cla = pred[0].size
    x_out = np.zeros((len(pred),(c_left+c_right+1)*cla)).astype(np.float32)
    valid = np.zeros(len(pred)).astype(np.int32)
    dogs = np.unique(dog_id_column_list)
    ids = np.array(range(len(pred)))
    if (c_left>0) & (c_right>0):        
        for i in dogs:
            act = ids[dog_id_column_list[:]==i][0]
            act_pred = pred[dog_id_column_list[:]==i]
            for k in range(c_left,len(act_pred)-c_right,1):
                x_out[act+k,:cla] = act_pred[k]
                x_out[act+k,cla:(1+c_left)*cla] = act_pred[(k-c_left):k].flatten()
                x_out[act+k,(1+c_left)*cla:] = act_pred[(k+1):k+c_right+1].flatten()
                valid[act+k]=1
#     else:
#         if (c_left==0) & (c_right>0):
#             for i in dogs:
#                 act = ids[dog_id_column_list[:]==i][0]
#                 act_pred = pred[dog_id_column_list[:]==i]
#                 for k in range(0,len(act_pred)-c_right,1):
#                     x_out[act+k,:cla] = act_pred[k]
#                     x_out[act+k,(1+c_left)*cla:] = act_pred[(k+1):k+c_right+1].flatten()
#                     valid[act+k]=1
#         if (c_left>0) & (c_right==0):
#             for i in dogs:
#                 act = ids[dog_id_column_list[:]==i][0]
#                 act_pred = pred[dog_id_column_list[:]==i]
#                 for k in range(c_left,len(act_pred)-1
#                                ,1):
#                     x_out[act+k,:cla] = act_pred[k]
#                     x_out[act+k,cla:(1+c_left)*cla] = act_pred[(k-c_left):k].flatten()
#                     valid[act+k]=1
#         if (c_left<1) & (c_right<1):
#             for i in dogs:
#                 act = ids[dog_id_column_list[:]==i][0]
#                 act_pred = pred[dog_id_column_list[:]==i]
#                 for k in range(0,len(act_pred)-1,1):
#                     x_out[act+k,:cla] = act_pred[k]
    return x_out,valid 

In [3]:
# already exists
fname_labels = "KH_labels_2020_dec_21.dms"
# fname_dogs = "dogs_2018_dec_28.dms"
path = "/Volumes/Gabor SSD/ML/gabor_venv/KH_2020_raw_asd/"
#output
fname_ts_base = "KH_2020_dec30_ts_lgbm_base.csv"
fname_ts_final = "KH_2020_dec30_ts_lgbm_final.csv"
fname_ts_base_turi = "KH_2020_dec30_ts_lgbm_base_turi.csv"

In [4]:
extension = 'csv'
os.chdir(path)
filenames = [i for i in glob.glob('*.{}'.format(extension))]
os.chdir("..")
conv=""

num=0
dog_names = []
dfs = []

for fn in filenames:
    fn = str(fn).split('.')[0]
    print("-> %s" %(fn))
    dfs.append(utils.convert_to_ts(fn,path,conv))
    num+=1
    dog_names.append(fn)
    print("finished: %s -> %d" %(fn,len(dfs[num-1])))

-> KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54
finished: KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54 -> 138000
-> KH-asdadhd-24_KH-asdadhd__2020-03-02-15-55-59
finished: KH-asdadhd-24_KH-asdadhd__2020-03-02-15-55-59 -> 165000
-> KH-asdadhd-14_KH-asdadhd__2020-01-16-16-00-32
SYNCUTC1 1579186833.613
finished: KH-asdadhd-14_KH-asdadhd__2020-01-16-16-00-32 -> 168000
-> KH-asdadhd-23_KH-asdadhd__2020-02-24-16-18-21
finished: KH-asdadhd-23_KH-asdadhd__2020-02-24-16-18-21 -> 84000
-> KH-asdadhd-20_KH-asdadhd__2020-02-21-08-17-32
finished: KH-asdadhd-20_KH-asdadhd__2020-02-21-08-17-32 -> 123000
-> KH-asdadhd-19_KH-asdadhd__2020-02-19-15-29-00
finished: KH-asdadhd-19_KH-asdadhd__2020-02-19-15-29-00 -> 189000
-> KH-asdadhd-22_KH-asdadhd__2020-02-24-12-01-03
SYNCUTC1 1582542064.302
finished: KH-asdadhd-22_KH-asdadhd__2020-02-24-12-01-03 -> 111000
-> KH-asdadhd-26_KH-asdadhd__2020-03-12-17-05-03
finished: KH-asdadhd-26_KH-asdadhd__2020-03-12-17-05-03 -> 99000
-> KH-asdadhd-2_KH-asdadhd__2019-1

In [5]:
df_all = dfs[0]

for k in range(0,len(dfs),1):
    df_all=df_all.append(other=dfs[k],ignore_index=True)
    print("%d %s ---> %d " %(k,str(dog_names[k]),len(dfs[k])))
print("Total ---> %d" %(len(df_all)))

0 KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54 ---> 138000 
1 KH-asdadhd-24_KH-asdadhd__2020-03-02-15-55-59 ---> 165000 
2 KH-asdadhd-14_KH-asdadhd__2020-01-16-16-00-32 ---> 168000 
3 KH-asdadhd-23_KH-asdadhd__2020-02-24-16-18-21 ---> 84000 
4 KH-asdadhd-20_KH-asdadhd__2020-02-21-08-17-32 ---> 123000 
5 KH-asdadhd-19_KH-asdadhd__2020-02-19-15-29-00 ---> 189000 
6 KH-asdadhd-22_KH-asdadhd__2020-02-24-12-01-03 ---> 111000 
7 KH-asdadhd-26_KH-asdadhd__2020-03-12-17-05-03 ---> 99000 
8 KH-asdadhd-2_KH-asdadhd__2019-11-07-15-11-48 ---> 72000 
9 KH-asdadhd-6_KH-asdadhd__2019-11-13-16-26-21 ---> 69000 
10 KH-asdadhd-27_KH-asdadhd__2020-07-13-14-10-09 ---> 198000 
11 KH-asdadhd-31_KH-asdadhd__2020-08-14-14-29-49 ---> 81000 
12 KH-asdadhd-16_KH-asdadhd__2020-01-21-11-08-25 ---> 96000 
13 KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36 ---> 99000 
Total ---> 1830000


In [6]:
df_all.head()

,name,timestamp,label,attitude_1,attitude_2,attitude_3,attitude_4,gravity_x,gravity_y,gravity_z,rotation_rate_x,rotation_rate_y,rotation_rate_z,user_acceleration_x,user_acceleration_y,user_acceleration_z
0,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.239,NaN,0.898376107215881,-0.146469369530678,0.414085865020752,0.000000000000000,0.744009673595428,0.263169169425964,-0.614159226417542,0.593363165855408,-0.311673462390900,-0.526831209659576,-0.039526641368866,-0.012162089347839,-0.062110304832458
1,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.245,NaN,0.900515496730804,-0.142615482211113,0.410708725452423,-0.007147963624448,0.737660348415375,0.262726366519928,-0.621958315372467,0.756584048271179,-0.262659043073654,-0.550245463848114,-0.052235484123230,-0.023346483707428,-0.061559081077576
2,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.254,NaN,0.902572870254517,-0.137624442577362,0.407667130231857,-0.015141053125262,0.731730997562408,0.260777205228806,-0.629734039306641,0.861211597919464,-0.229370266199112,-0.566556513309479,-0.049373209476471,-0.023304671049118,-0.048946380615234
3,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.274,NaN,0.904587209224701,-0.132073223590851,0.404641151428223,-0.023331156000495,0.725903570652008,0.257824987173080,-0.637644410133362,0.862933933734894,-0.235744506120682,-0.548381745815277,-0.043133795261383,-0.037976354360580,-0.043385863304138
4,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.294,NaN,0.906427502632141,-0.126911014318466,0.401625305414200,-0.031303085386753,0.720143020153046,0.255215495824814,-0.645181417465210,0.783847093582153,-0.226670056581497,-0.553145110607147,-0.050435006618500,-0.076916605234146,-0.059454321861267


In [7]:
df_all.to_csv(fname_ts_base)

In [8]:
labels,label_dict,dog_id_column_list,dog_dict = utils.make_labels_dogids(df_all,labels_dict_fname=fname_labels,dog_names=dog_names)
df_all['dog_id'] = dog_id_column_list
df_all['label_id'] = labels
df_all.head()
cols = df_all.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
df_all = df_all[cols]

[0 0 0 ... 0 0 0]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
{'KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54': 0, 'KH-asdadhd-24_KH-asdadhd__2020-03-02-15-55-59': 1, 'KH-asdadhd-14_KH-asdadhd__2020-01-16-16-00-32': 2, 'KH-asdadhd-23_KH-asdadhd__2020-02-24-16-18-21': 3, 'KH-asdadhd-20_KH-asdadhd__2020-02-21-08-17-32': 4, 'KH-asdadhd-19_KH-asdadhd__2020-02-19-15-29-00': 5, 'KH-asdadhd-22_KH-asdadhd__2020-02-24-12-01-03': 6, 'KH-asdadhd-26_KH-asdadhd__2020-03-12-17-05-03': 7, 'KH-asdadhd-2_KH-asdadhd__2019-11-07-15-11-48': 8, 'KH-asdadhd-6_KH-asdadhd__2019-11-13-16-26-21': 9, 'KH-asdadhd-27_KH-asdadhd__2020-07-13-14-10-09': 10, 'KH-asdadhd-31_KH-asdadhd__2020-08-14-14-29-49': 11, 'KH-asdadhd-16_KH-asdadhd__2020-01-21-11-08-25': 12, 'KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36': 13}
                                         dog_name  dog_id
0   KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54       0
1   KH-asdadhd-24_KH-asdadhd__2020-03-02-15-55-59       1
2   KH-asdadhd-14_KH-asdadhd__2020-01-16-16-0

In [9]:
df_all = utils.clearNaN(df_all)
df_all.to_csv(fname_ts_final)
df_all

,dog_id,label_id,name,timestamp,label,attitude_1,attitude_2,attitude_3,attitude_4,gravity_x,gravity_y,gravity_z,rotation_rate_x,rotation_rate_y,rotation_rate_z,user_acceleration_x,user_acceleration_y,user_acceleration_z
0,0,-1,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.239,unknown,0.898376107215881,-0.146469369530678,0.414085865020752,0.000000000000000,0.744009673595428,0.263169169425964,-0.614159226417542,0.593363165855408,-0.311673462390900,-0.526831209659576,-0.039526641368866,-0.012162089347839,-0.062110304832458
1,0,-1,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.245,unknown,0.900515496730804,-0.142615482211113,0.410708725452423,-0.007147963624448,0.737660348415375,0.262726366519928,-0.621958315372467,0.756584048271179,-0.262659043073654,-0.550245463848114,-0.052235484123230,-0.023346483707428,-0.061559081077576
2,0,-1,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.254,unknown,0.902572870254517,-0.137624442577362,0.407667130231857,-0.015141053125262,0.731730997562408,0.260777205228806,-0.629734039306641,0.861211597919464,-0.229370266199112,-0.566556513309479,-0.049373209476471,-0.023304671049118,-0.048946380615234
3,0,-1,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.274,unknown,0.904587209224701,-0.132073223590851,0.404641151428223,-0.023331156000495,0.725903570652008,0.257824987173080,-0.637644410133362,0.862933933734894,-0.235744506120682,-0.548381745815277,-0.043133795261383,-0.037976354360580,-0.043385863304138
4,0,-1,KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54,1580720515.294,unknown,0.906427502632141,-0.126911014318466,0.401625305414200,-0.031303085386753,0.720143020153046,0.255215495824814,-0.645181417465210,0.783847093582153,-0.226670056581497,-0.553145110607147,-0.050435006618500,-0.076916605234146,-0.059454321861267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829995,13,42,KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36,1605000513.033,Snack_eat,-0.940424561500549,0.315202742815018,0.112151898443699,-0.060587670654058,-0.172746002674103,0.606438875198364,-0.776138365268707,0.303315222263336,0.146527364850044,0.479583144187927,0.090075209736824,0.051931381225586,0.086640715599060
1829996,13,42,KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36,1605000513.052,Snack_eat,-0.941047787666321,0.313922822475433,0.109058208763599,-0.063148140907288,-0.165610685944557,0.604606389999390,-0.779117524623871,0.202920973300934,-0.001860240707174,0.204875424504280,0.097878992557526,0.046000719070435,0.061473429203033
1829997,13,42,KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36,1605000513.072,Snack_eat,-0.941639482975006,0.312344729900360,0.107716217637062,-0.064443983137608,-0.162602216005325,0.602115571498871,-0.781675994396210,0.179568782448769,0.093581244349480,0.117557823657990,0.041061848402023,0.053181171417236,0.027232944965363
1829998,13,42,KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36,1605000513.091,Snack_eat,-0.942188799381256,0.310907363891602,0.106581829488277,-0.065246708691120,-0.160269051790237,0.599775075912476,-0.783953845500946,0.137468472123146,0.096682399511337,0.060601331293583,0.018033012747765,0.019332587718964,0.027287662029266


In [10]:
data_filt,labels,cat_inv,dog_fold,dog_occ,dog_inv,dog_fold_inv = utils.get_raw_ts(fname_labels=fname_labels,fname_data=fname_ts_final)

/Volumes/Gabor SSD/ML/gabor_venv/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
data_filt
data_filt.to_csv(fname_ts_base_turi)

In [12]:
ts_raw = pd.read_csv(fname_ts_final,sep=",")

# FEATURES

# Bout length

In [25]:
# behaviour elements' bout-lengths collection
labels = ts_raw['label_id'].values.astype(np.int32)
label_names_sequence = ts_raw['label'].values
list_labels = np.unique(labels)
stat_labels = []
labels_inv = {}

for k in range(len(list_labels)):
    stat_labels.append([])
    labels_inv[list_labels[k]]=k
    
last = labels[0]
n = 0

for k in range(len(labels)):
    act = labels[k]
#     print("%dth -> %d and %s (%d)" %(k, act,label_names_sequence[k],last),end=" ") #,stat_labels[labels_inv[last]])
    if act == last:
        n+=1
    else:
        stat_labels[labels_inv[last]].append(n)
#         print("done: %d cat %s"% (last,stat_labels[labels_inv[last]]))
        n=0
    last = act

label_names = pd.read_csv(fname_labels,sep=" ")

In [26]:
for k in range(len(stat_labels)):
    #plt.figure(figsize=(20,10))
    if list_labels[k]==-1:
        print(np.median(stat_labels[k]),np.max(stat_labels[k]),np.mean(stat_labels[k]),np.std(stat_labels[k]),list_labels[k],"na") #,bins=100,label=str(list_labels[k]),alpha=1)
    else:
        print(np.median(stat_labels[k]),np.max(stat_labels[k]),np.mean(stat_labels[k]),np.std(stat_labels[k]),list_labels[k],label_names[label_names['label_id']==list_labels[k]].values) #,bins=100,label=str(list_labels[k]),alpha=1)        
    #plt.legend()
    #plt.show()

3.0 14770 104.47828009284846 634.1602090304864 -1 na
5.0 961 43.909774436090224 148.5829030453798 1 [[1 'Spider']]
5.0 14942 258.1025641025641 1693.046499957714 2 [[2 'Crab']]
5.0 54 13.025641025641026 12.542210126030277 3 [[3 'Rabbit']]
5.0 23082 265.0625 2341.119131475461 4 [[4 'Frog']]
5.0 1157 20.032178217821784 93.62518201390348 5 [[5 'Bear']]
5.0 473 22.65714285714286 71.44799354994461 6 [[6 'Seal']]
5.0 1661 22.759615384615383 129.23673828648336 7 [[7 'Dwarf']]
5.0 44897 276.28115942028984 3407.9376532568212 8 [[8 'Goliath']]
5.0 875 25.1875 101.43562704818801 9 [[9 'Swimming']]
5.0 744 15.938366718027735 48.12092393109905 10 [[10 'Hopscotch']]
5.0 376 27.24 74.57896754447596 11 [[11 'Shoe_off_same']]
5.0 256 40.0625 71.54322884627167 12 [[12 'Shoe_off_other']]
5.0 382 8.747747747747749 35.963812760338605 13 [[13 'Sock_off_same']]
5.0 271 13.192307692307692 43.40331361930901 14 [[14 'Sock_off_other']]
5.0 965 9.766968325791856 59.817841845273435 15 [[15 'Sock_on_same']]
5.0 2597

# feature function

In [15]:
def stat_features_new(data_filt,cat_inv,dog_inv,str_t=100,cat_pol='last',window=32,shift=16,verbose=False):
    
    ndogs = data_filt['name'].value_counts().index
    
    print("actual window(act_w): %d shift: %d str_t: %d "%(window,shift,str_t))
    x_all = []
    y_all=[]
    x_t_all=[]
    #cat_hist = np.zeros((ndogs,len(cat_inv))).astype(np.int32)
    #cat_hist_all = []
    ts_str = []
    ts_end = []
    
    for i in range(len(ndogs)):
        data = data_filt[data_filt['name']==ndogs[i]]
        dogid = dog_inv[ndogs[i]]
        nstat_feat = 2*5
        nts = len(data.head(1).values[0,6:])
        dim = nstat_feat*nts
        ts_len = int((len(data)-window-str_t)/shift+1)

        if verbose==True:
            print("%dth dog samples: %d dimension: %d (data class: %d) name: %s"%(i,ts_len,dim,nts,ndogs[i]))
        
        c = dogid*np.ones((ts_len,1)).astype(np.int32)
        c_str = []
        y = np.zeros((ts_len,2)).astype(np.int32)
        x = np.zeros((ts_len,dim)).astype(np.float32)
        x_t = np.zeros((ts_len,2)).astype(np.int32)

        if cat_pol=='last':
            cat_idx = 0
        if cat_pol=='first':
            cat_idx = 1-window

        act_y = data['label_id'].values
        act_x = data.values[:,6:].astype(np.float32)
        ts_act = data['timestamp'].values
        act_grad_x = np.zeros((len(act_x),len(act_x[0,:])))
        act_grad_x[1:] = act_x[1:] - act_x[:-1]
        #print("diff: %f %s and %s" %(np.linalg.norm(act_grad_x-act_x),act_x.shape,act_grad_x.shape))
        
        #cat_hist = np.zeros((ts_len,len(cat_inv)+1)).astype(np.int32)

        for k in range(ts_len):
            str_x = k*shift+str_t
            act = act_x[str_x-window:str_x]
            act_grad = act_grad_x[str_x-window:str_x]
            act_sss = np.unique(act_y[str_x-window:str_x],return_counts=True)
            y[k,0] = act_y[str_x] 
            y[k,1] = act_sss[0][np.argsort(-act_sss[1])[0]] #int(cat_inv[act_y[str_x+cat_idx]])
            x_t[k,0] = ts_act[str_x-window:str_x][0]
            x_t[k,1] = ts_act[str_x-window:str_x][-1]
            
            #print("\tact diff:  %f %s vs %s" %(np.linalg.norm(act_grad-act),act.shape,act_grad.shape))
            
            #for l in range(window):
            #    act_sss = np.unique(act_y,return_counts=True)
            #    act_cat = act_sss[0][np.argsort(-act_sss[1])[0]]
            #    cat_hist[k,act_cat]+=1
            x[k,:nts] = np.min(act,axis=0)
            x[k,nts:2*nts] = np.max(act,axis=0)
            x[k,2*nts:3*nts] = np.mean(act,axis=0)
            x[k,3*nts:4*nts] = np.std(act,axis=0)
            x[k,4*nts:5*nts] = scipy.stats.skew(act,axis=0)
            x[k,5*nts:6*nts] = np.min(act_grad,axis=0)
            x[k,6*nts:7*nts] = np.max(act_grad,axis=0)
            x[k,7*nts:8*nts] = np.mean(act_grad,axis=0)
            x[k,8*nts:9*nts] = np.std(act_grad,axis=0)
            x[k,9*nts:10*nts] = scipy.stats.skew(act_grad,axis=0)
            #print("\t %dth dog %dth td diff mean: %f" % (i,k,np.linalg.norm(x[k,7*nts:8*nts]-x[k,2*nts:3*nts])))
            #print("\t\tafter diff: %f %s and %s" %(np.linalg.norm(act_grad_x-act_x),act_x.shape,act_grad_x.shape))
        x_all.append(np.concatenate((x_t,c,y,x),axis=1))
        
        #cat_hist_all.append(cat_hist)
    full = x_all[0]
    for k in range(len(x_all)-1):
        full = np.concatenate((full,x_all[k+1]),axis=0).astype(np.float32)
    full = np.nan_to_num(np.array(full).astype(np.float32))
    return full#,cat_hist_all

# Feature calculation

In [16]:
feats = []
act_window_list = [60]#[15, 32, 60, 81, 100, 120, 149]
for k in range(1): #len(stat_labels)):
    #act_window = min(np.median(stat_labels[k]),149)
    act_window = act_window_list[k]
    if act_window>=150:
        act_window=149
    x_all = stat_features_new(data_filt,cat_inv,dog_inv,str_t=150,window=int(act_window),shift=5,verbose=True)
    feats.append(x_all)
    np.savetxt(str("feat_cat_"+str(k)),x_all)

actual window(act_w): 15 shift: 5 str_t: 150 
0th dog samples: 55168 dimension: 150 (data class: 15) name: KH-asdadhd-18_KH-asdadhd__2020-02-03-10-01-54


/Volumes/Gabor SSD/ML/gabor_venv/lib/python3.8/site-packages/numpy/core/_methods.py:202: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Volumes/Gabor SSD/ML/gabor_venv/lib/python3.8/site-packages/scipy/stats/stats.py:1082: RuntimeWarning: invalid value encountered in subtract
  a_zero_mean = a - np.expand_dims(np.mean(a, axis), axis)
/Volumes/Gabor SSD/ML/gabor_venv/lib/python3.8/site-packages/numpy/core/_methods.py:160: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/Volumes/Gabor SSD/ML/gabor_venv/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)


1th dog samples: 39568 dimension: 150 (data class: 15) name: KH-asdadhd-27_KH-asdadhd__2020-07-13-14-10-09
2th dog samples: 37768 dimension: 150 (data class: 15) name: KH-asdadhd-19_KH-asdadhd__2020-02-19-15-29-00
3th dog samples: 33568 dimension: 150 (data class: 15) name: KH-asdadhd-14_KH-asdadhd__2020-01-16-16-00-32
4th dog samples: 32968 dimension: 150 (data class: 15) name: KH-asdadhd-24_KH-asdadhd__2020-03-02-15-55-59
5th dog samples: 24568 dimension: 150 (data class: 15) name: KH-asdadhd-20_KH-asdadhd__2020-02-21-08-17-32
6th dog samples: 22168 dimension: 150 (data class: 15) name: KH-asdadhd-22_KH-asdadhd__2020-02-24-12-01-03
7th dog samples: 19768 dimension: 150 (data class: 15) name: KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36
8th dog samples: 19768 dimension: 150 (data class: 15) name: KH-asdadhd-26_KH-asdadhd__2020-03-12-17-05-03
9th dog samples: 19168 dimension: 150 (data class: 15) name: KH-asdadhd-16_KH-asdadhd__2020-01-21-11-08-25
10th dog samples: 16768 dimension: 15

6th dog samples: 22147 dimension: 150 (data class: 15) name: KH-asdadhd-22_KH-asdadhd__2020-02-24-12-01-03
7th dog samples: 19747 dimension: 150 (data class: 15) name: KH-asdadhd-35_KH-asdadhd__2020-11-10-09-56-36
8th dog samples: 19747 dimension: 150 (data class: 15) name: KH-asdadhd-26_KH-asdadhd__2020-03-12-17-05-03
9th dog samples: 19147 dimension: 150 (data class: 15) name: KH-asdadhd-16_KH-asdadhd__2020-01-21-11-08-25
10th dog samples: 16747 dimension: 150 (data class: 15) name: KH-asdadhd-23_KH-asdadhd__2020-02-24-16-18-21
11th dog samples: 16147 dimension: 150 (data class: 15) name: KH-asdadhd-31_KH-asdadhd__2020-08-14-14-29-49
12th dog samples: 14347 dimension: 150 (data class: 15) name: KH-asdadhd-2_KH-asdadhd__2019-11-07-15-11-48
13th dog samples: 13747 dimension: 150 (data class: 15) name: KH-asdadhd-6_KH-asdadhd__2019-11-13-16-26-21
actual window(act_w): 149 shift: 5 str_t: 150 
0th dog samples: 55141 dimension: 150 (data class: 15) name: KH-asdadhd-18_KH-asdadhd__2020-02-

In [17]:
print(feats[0].shape)
#print(feats[1].shape)
#print(feats[2].shape)
#feats.head()

(365552, 155)


# TRAIN ML model

In [42]:
import time
from datetime import datetime 
start_time = datetime.now()
dbg = open("jan03_2020_results_raw_category_KH.txt","w")

res = open("jan03_2020_results_all_KH.txt","w")

print("run,window_size,fold_id,category_id,acc_vl,auc_vl,prec,recall,true_pos,pos_predicted,num_pos,label,bout_median",file=res)

print('Time start (hh:mm:ss.ms) {}'.format(datetime.now()))
print('Time start (hh:mm:ss.ms) {}'.format(datetime.now()),file=dbg)

i=0
n_folds = 3

label_lists = pd.read_csv(fname_labels,delimiter=" ")
for run_exp in range(10):
    for length_cat in range(1):#,3,5]: #np.unique(y_full_pop):
        data_val = np.loadtxt(str("feat_cat_val_"+str(length_cat)))
        data = np.loadtxt(str("feat_cat_"+str(length_cat)))#(str("data/feat_cat_"+str(length_cat)))  ## 0 -> 81, 3 -> 149, 5 -> 32
        print("window length: %d" % act_window_list[length_cat])
        print("window length: %d" % act_window_list[length_cat],file=dbg)
        print("data: %s" % (str(data.shape)))
        print("data: %s" % (str(data.shape)),file=dbg)
        print("test data: %s" % (str(data_val.shape)))
        print("test data: %s" % (str(data_val.shape)),file=dbg)
        dbg.flush()
        x_full_base = data[:,5:].astype(np.float32)#take all from the 6. col or all except first 5 col
        y_full_last_base = data[:,3].astype(np.int32)#take just the 4. col
        y_full_pop_base = data[:,4].astype(np.int32)#take just the 5. col
        #test data
        x_val_full_base = data_val[:,5:].astype(np.float32)#take all from the 6. col or all except first 5 col
        y_val_full_last_base = data_val[:,3].astype(np.int32)#take just the 4. col
        y_val_full_pop_base = data_val[:,4].astype(np.int32)#take just the 5. col
        #cleaning up the unknown labels
        x_full = x_full_base[y_full_last_base[:]>=0]
        y_full_last = y_full_last_base[y_full_last_base[:]>=0]
        dog_id = data[:,2].astype(np.int32)[y_full_last_base[:]>=0]
        dog_id_list = np.unique(data[:,2].astype(np.int32)) #feats[0][:,2].astype(np.int32))
        folds = np.zeros(len(dog_id)).astype(np.int32)
        #cleaning up the unknown labels in validation data
        x_val_full = x_val_full_base[y_val_full_last_base[:]>=0]
        y_val_full_last = y_val_full_last_base[y_val_full_last_base[:]>=0]
        dog_id_val = data_val[:,2].astype(np.int32)[y_val_full_last_base[:]>=0]
        dog_id_list_val = np.unique(data_val[:,2].astype(np.int32)) #feats[0][:,2].astype(np.int32))
        folds_val = np.zeros(len(dog_id_val)).astype(np.int32)
        #folds_list = np.array([0,1,2]).astype(np.int32) 
        #folds_list = np.random.randint(0,n_folds,size=len(dog_id_list))
        tr,te =int(round(len(dog_id_list)*0.7)),int(round(len(dog_id_list)*0.3))
        train = [0] * tr
        test = [2] * te
        validation =[1] * (len(dog_id_list_val))
        #folds_list = train+validation+test
        folds_list = train+test
        shuffle(folds_list)
        print(folds_list)
        for k in range(len(dog_id_list)):
            folds[dog_id[:]==dog_id_list[k]]=folds_list[k]
        folds_val.fill(1)
        for fold in range(n_folds):
            tr_fold = 0 #0
            #vl_fold = fold #1
            vl_fold = 1 #1
            te_fold = 2 #2
            #x_tr = x_full[folds[:]!=vl_fold]
            x_tr = x_full[folds[:]==tr_fold]
            x_vl = x_val_full[folds_val[:]==vl_fold]
            #x_te = x_full[folds[:]==te_fold] 
            pred_tr = np.zeros((len(x_tr),len(np.unique(y_full_last)))).astype(np.float32)
            pred_vl = np.zeros((len(x_vl),len(np.unique(y_val_full_last)))).astype(np.float32)
            #pred_te = np.zeros((len(x_te),len(np.unique(y_full_last)))).astype(np.float32)
            y_vl_all = np.zeros((len(x_vl),len(np.unique(y_val_full_last)))).astype(np.float32)
            cat_id = 0
            cat_list = np.zeros(len(np.unique(y_full_last)))
            auc_sum=0
            acc_sum=0
            n=0
            print(folds_val)
            for cat in np.unique(y_full_last):
                if cat!=-1:
                    y_raw = (y_full_last[y_full_last[:]>=0][:]==cat).astype(np.int32)
                    y_val_raw = (y_val_full_last[y_val_full_last[:]>=0][:]==cat).astype(np.int32)
                    #y_tr = y_raw[folds[:]!=vl_fold]
                    y_tr = y_raw[folds[:]==tr_fold]
                    y_vl = y_val_raw[folds_val[:]==vl_fold]
                    #y_te = y_raw[folds[:]==te_fold]
                    dog_tr = dog_id[folds[:]==tr_fold]
                    dog_vl = dog_id_val[folds_val[:]==vl_fold]
                    #dog_te = dog_id[folds[:]==te_fold]
                    #print("category: %d %s window: %d",cat,label_lists[label_lists['label_id']==cat]['label'].values,act_window_list[length_cat])
                    #print("category: %d %s window: %d",cat,label_lists[label_lists['label_id']==cat]['label'].values,act_window_list[length_cat],file=dbg)
                    print("category: %d %s " % (cat,label_lists[label_lists['label_id']==cat]['label'].values))
                    print("category: %d %s " % (cat,label_lists[label_lists['label_id']==cat]['label'].values),file=dbg)
                    for n_est in [100]: #[20,50,100,200,500,1000]: #[1,3,5,10,20,50,100]:
                        for max_depth in [-1]: #[7][3,5,7,10,20,30]: #[1,2,3,5,7,10,20]:
                            if (np.sum(y_tr)>0) & (np.sum(y_vl)>0):
                                print("\tcategory: %d %s %d-th fold train set: %s -> %d positive %d negative sample" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,i,x_tr.shape, len(y_tr[y_tr[:]==1]),len(y_tr[y_tr[:]==0])))
                                print("\tcategory: %d %s test set: %s -> %d positive %d negative sample" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,x_vl.shape, len(y_vl[y_vl[:]==1]),len(y_vl[y_vl[:]==0])))
                                print("\tcategory: %d %s %d-th fold train set: %s -> %d positive %d negative sample" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,i,x_tr.shape, len(y_tr[y_tr[:]==1]),len(y_tr[y_tr[:]==0])),file=dbg)
                                print("\tcategory: %d %s test set: %s -> %d positive %d negative sample" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,x_vl.shape, len(y_vl[y_vl[:]==1]),len(y_vl[y_vl[:]==0])),file=dbg)
                                dbg.flush()

                                #cf = ens.GradientBoostingClassifier(n_estimators=n_est,max_depth=max_depth)
                                #cf = ens.AdaBoostClassifier(n_estimators=n_est) 
                                #cf = xgb.XGBClassifier(n_estimators=n_est,max_depth=max_depth) #lin.LogisticRegression(solver="newton-cg",max_iter=max_iter,C=c,n_jobs=-1)
                                #cf = xgb.XGBClassifier(eta=0.3,max_depth=max_depth,max_delta_step=4, scale_pos_weight = 1, min_child_weight = 1 )
                                cf = lgb.LGBMClassifier(n_estimators=n_est,max_depth=max_depth)
                                cf.fit(x_tr,y_tr)

                                pred = cf.predict(x_vl)
                                pred_pr = cf.predict_proba(x_vl)

                                if np.count_nonzero(y_vl) > 1:
                                    acc_vl = met.accuracy_score(y_true=y_vl,y_pred=pred)
                                if np.count_nonzero(y_vl) > 1:
                                    auc_vl = met.roc_auc_score(y_true=y_vl,y_score=pred_pr[:,1])
                                cat_list[cat_id] = cat
                                y_vl_all[:,cat_id]=y_vl
                                #np.savetxt("pred/pred_"+str(length_cat)+"_"+str(fold)+"_"+str(cat)+".gt",y_vl)
                                if auc_vl<0.5:
                                    auc_vl=1-auc_vl
                                    pred_vl[:,cat_id]=pred_pr[:,0]
                                    #np.savetxt("pred/pred_"+str(length_cat)+"_"+str(fold)+"_"+str(cat)+".pred",pred_pr[:,0])
                                else:
                                    pred_vl[:,cat_id]=pred_pr[:,1]
                                    #np.savetxt("pred/pred_"+str(length_cat)+"_"+str(fold)+"_"+str(cat)+".pred",pred_pr[:,1])

    #                             pred = cf.predict(x_te)
    #                             pred_pr = cf.predict_proba(x_te)
    #                             if np.count_nonzero(y_te) > 1:
    #                                 acc = met.accuracy_score(y_true=y_te,y_pred=pred)
    #                             if np.count_nonzero(y_te) > 1:
    #                                 auc = met.roc_auc_score(y_true=y_te,y_score=pred_pr[:,1])
    #                             cat_list[cat_id] = cat
    #                             if auc<0.5:
    #                                 auc=1-auc
    #                                 pred_te[:,cat_id]=pred_pr[:,0]
    #                             else:
    #                                 pred_te[:,cat_id]=pred_pr[:,1]
                                print(confusion_matrix(y_vl, pred))
                                print(confusion_matrix(y_vl, pred),file=dbg)
                                tp = 0
                                tn = 0 
                                num = 0
                                pos = y_vl[y_vl[:]==1].size
                                neg = y_vl[y_vl[:]==0].size
                                for k in range(len(pred)):
                                    if (pred[k]==1) & (y_vl[k]==1):
                                        tp+=1
                                    if (pred[k]==0) & (y_vl[k]==0):
                                        tn+=1
                                #acc = (tp+tn)/(pos+neg)
                                pred_pos = len(pred[pred[:]==1])
                                if pred_pos<1:
                                    pred_pos=1
                                acc_sum+=acc_vl
                                auc_sum+=auc_vl
                                n+=1
                                print("\tcategory: %d %s n_est: %d max_depth: %d -> acc_vl: %f auc_vl: %f" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,n_est,max_depth,acc_vl,auc_vl))
                                print("\tcategory: %d %s n_est: %d max_depth: %d -> acc_vl: %f auc_vl: %f" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,n_est,max_depth,acc_vl,auc_vl),file=dbg)
                                print("\tcategory: %d %s prec: %f (%d out of %d) recall: %f (%d out of %d)" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,tp/pred_pos,tp,len(pred[pred[:]==1]),tp/pos,tp,pos))
                                print("\tcategory: %d %s prec: %f (%d out of %d) recall: %f (%d out of %d)" % (cat,label_lists[label_lists['label_id']==cat]['label'].values,tp/pred_pos,tp,len(pred[pred[:]==1]),tp/pos,tp,pos),file=dbg)
                                dbg.flush()

#                                 print(length_cat,act_window_list[length_cat],fold,cat,"acc_vl:",acc_vl,"auc_vl:",auc_vl,"prec:",tp/pred_pos,"recall:",tp/pos,"tp:",tp,"pred_pos:",len(pred[pred[:]==1]),"pos:",pos,"label:",label_lists[label_lists['label_id']==cat]['label'].values[0],file=res)
#                                 res.flush()
                                
                                print(f"{run_exp},{act_window_list[length_cat]},{fold},{cat},{acc_vl},{auc_vl},{tp/pred_pos},{tp/pos},{tp},{len(pred[pred[:]==1])},{pos},{label_lists[label_lists['label_id']==cat]['label'].values[0]}",file=res)
                                res.flush()

#             np.savetxt("pred/pred_"+str(run_exp)+"_"+str(length_cat)+"_"+str(fold)+"_all.pred",pred_vl)
#             np.savetxt("pred/pred_"+str(run_exp)+"_"+str(length_cat)+"_"+str(fold)+"_all.gt",y_vl_all)

            cat_id+=1
            print("summary: acc %f auc %f (%d)" % (acc_sum/n,auc_sum/n,n))
            print("summary: acc %f auc %f (%d)" % (acc_sum/n,auc_sum/n,n),file=dbg)
            dbg.flush()

    time.sleep(3) 

    print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))
    print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time),file=dbg)
    print('Time end (hh:mm:ss.ms) {}'.format(datetime.now()))         
    print('Time end (hh:mm:ss.ms) {}'.format(datetime.now()),file=dbg)  
dbg.close() 

Time start (hh:mm:ss.ms) 2021-01-03 19:32:49.080379
window length: 15
data: (674042, 155)
test data: (192872, 155)
[0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0]
[1 1 1 ... 1 1 1]
category: 1 ['Spider'] 
	category: 1 ['Spider'] 0-th fold train set: (252703, 150) -> 2113 positive 250590 negative sample
	category: 1 ['Spider'] test set: (109185, 150) -> 707 positive 108478 negative sample
[[108139    339]
 [   562    145]]
	category: 1 ['Spider'] n_est: 100 max_depth: -1 -> acc_vl: 0.991748 auc_vl: 0.981097
	category: 1 ['Spider'] prec: 0.299587 (145 out of 484) recall: 0.205092 (145 out of 707)
category: 2 ['Crab'] 
	category: 2 ['Crab'] 0-th fold train set: (252703, 150) -> 1832 positive 250871 negative sample
	category: 2 ['Crab'] test set: (109185, 150) -> 742 positive 108443 negative sample
[[108195    248]
 [   584    158]]
	category: 2 ['Crab'] n_est: 100 max_depth: -1 -> acc_vl: 0.992380 auc_vl: 0.989321
	categor

[[75546    50]
 [31874  1715]]
	category: 19 ['Drawing'] n_est: 100 max_depth: -1 -> acc_vl: 0.707616 auc_vl: 0.605204
	category: 19 ['Drawing'] prec: 0.971671 (1715 out of 1765) recall: 0.051058 (1715 out of 33589)
category: 20 ['Ball'] 
	category: 20 ['Ball'] 0-th fold train set: (252703, 150) -> 15900 positive 236803 negative sample
	category: 20 ['Ball'] test set: (109185, 150) -> 948 positive 108237 negative sample
[[93006 15231]
 [  457   491]]
	category: 20 ['Ball'] n_est: 100 max_depth: -1 -> acc_vl: 0.856317 auc_vl: 0.835935
	category: 20 ['Ball'] prec: 0.031230 (491 out of 15722) recall: 0.517932 (491 out of 948)
category: 21 ['Book'] 
	category: 21 ['Book'] 0-th fold train set: (252703, 150) -> 1414 positive 251289 negative sample
	category: 21 ['Book'] test set: (109185, 150) -> 901 positive 108284 negative sample
[[107936    348]
 [   838     63]]
	category: 21 ['Book'] n_est: 100 max_depth: -1 -> acc_vl: 0.989138 auc_vl: 0.950305
	category: 21 ['Book'] prec: 0.153285 (63 

[[108531     26]
 [   612     16]]
	category: 38 ['Knee_same'] n_est: 100 max_depth: -1 -> acc_vl: 0.994157 auc_vl: 0.512592
	category: 38 ['Knee_same'] prec: 0.380952 (16 out of 42) recall: 0.025478 (16 out of 628)
category: 39 ['Knee_other'] 
	category: 39 ['Knee_other'] 0-th fold train set: (252703, 150) -> 232 positive 252471 negative sample
	category: 39 ['Knee_other'] test set: (109185, 150) -> 309 positive 108876 negative sample
[[108795     81]
 [   300      9]]
	category: 39 ['Knee_other'] n_est: 100 max_depth: -1 -> acc_vl: 0.996511 auc_vl: 0.514124
	category: 39 ['Knee_other'] prec: 0.100000 (9 out of 90) recall: 0.029126 (9 out of 309)
category: 40 ['Pray'] 
	category: 40 ['Pray'] 0-th fold train set: (252703, 150) -> 2922 positive 249781 negative sample
	category: 40 ['Pray'] test set: (109185, 150) -> 327 positive 108858 negative sample
[[108317    541]
 [   293     34]]
	category: 40 ['Pray'] n_est: 100 max_depth: -1 -> acc_vl: 0.992362 auc_vl: 0.862859
	category: 40 ['P

[[108812    233]
 [   137      3]]
	category: 14 ['Sock_off_other'] n_est: 100 max_depth: -1 -> acc_vl: 0.996611 auc_vl: 0.549719
	category: 14 ['Sock_off_other'] prec: 0.012712 (3 out of 236) recall: 0.021429 (3 out of 140)
category: 15 ['Sock_on_same'] 
	category: 15 ['Sock_on_same'] 0-th fold train set: (252703, 150) -> 925 positive 251778 negative sample
	category: 15 ['Sock_on_same'] test set: (109185, 150) -> 798 positive 108387 negative sample
[[108071    316]
 [   778     20]]
	category: 15 ['Sock_on_same'] n_est: 100 max_depth: -1 -> acc_vl: 0.989980 auc_vl: 0.849576
	category: 15 ['Sock_on_same'] prec: 0.059524 (20 out of 336) recall: 0.025063 (20 out of 798)
category: 16 ['Sock_on_other'] 
	category: 16 ['Sock_on_other'] 0-th fold train set: (252703, 150) -> 1012 positive 251691 negative sample
	category: 16 ['Sock_on_other'] test set: (109185, 150) -> 782 positive 108403 negative sample
[[108231    172]
 [   764     18]]
	category: 16 ['Sock_on_other'] n_est: 100 max_depth:

[[109049     27]
 [   109      0]]
	category: 33 ['Puding_open'] n_est: 100 max_depth: -1 -> acc_vl: 0.998754 auc_vl: 0.500307
	category: 33 ['Puding_open'] prec: 0.000000 (0 out of 27) recall: 0.000000 (0 out of 109)
category: 34 ['Puding_eat'] 
	category: 34 ['Puding_eat'] 0-th fold train set: (252703, 150) -> 23606 positive 229097 negative sample
	category: 34 ['Puding_eat'] test set: (109185, 150) -> 291 positive 108894 negative sample
[[107499   1395]
 [   287      4]]
	category: 34 ['Puding_eat'] n_est: 100 max_depth: -1 -> acc_vl: 0.984595 auc_vl: 0.764163
	category: 34 ['Puding_eat'] prec: 0.002859 (4 out of 1399) recall: 0.013746 (4 out of 291)
category: 35 ['Clapping'] 
	category: 35 ['Clapping'] 0-th fold train set: (252703, 150) -> 412 positive 252291 negative sample
	category: 35 ['Clapping'] test set: (109185, 150) -> 472 positive 108713 negative sample
[[108595    118]
 [   354    118]]
	category: 35 ['Clapping'] n_est: 100 max_depth: -1 -> acc_vl: 0.995677 auc_vl: 0.634

...

	category: 9 ['Swimming'] n_est: 100 max_depth: -1 -> acc_vl: 0.935440 auc_vl: 0.774779
	category: 9 ['Swimming'] prec: 0.048077 (325 out of 6760) recall: 0.346113 (325 out of 939)
category: 10 ['Hopscotch'] 
	category: 10 ['Hopscotch'] 0-th fold train set: (250449, 150) -> 1532 positive 248917 negative sample
	category: 10 ['Hopscotch'] test set: (109185, 150) -> 1110 positive 108075 negative sample
[[106731   1344]
 [   558    552]]
	category: 10 ['Hopscotch'] n_est: 100 max_depth: -1 -> acc_vl: 0.982580 auc_vl: 0.884459
	category: 10 ['Hopscotch'] prec: 0.291139 (552 out of 1896) recall: 0.497297 (552 out of 1110)
category: 11 ['Shoe_off_same'] 
	category: 11 ['Shoe_off_same'] 0-th fold train set: (250449, 150) -> 141 positive 250308 negative sample
	category: 11 ['Shoe_off_same'] test set: (109185, 150) -> 87 positive 109098 negative sample
[[108887    211]
 [    87      0]]
	category: 11 ['Shoe_off_same'] n_est: 100 max_depth: -1 -> acc_vl: 0.997271 auc_vl: 0.501164
	category: 11 

[[108337    278]
 [   536     34]]
	category: 28 ['Toothbrush_same'] n_est: 100 max_depth: -1 -> acc_vl: 0.992545 auc_vl: 0.540190
	category: 28 ['Toothbrush_same'] prec: 0.108974 (34 out of 312) recall: 0.059649 (34 out of 570)
category: 29 ['Toothbrush_other'] 
	category: 29 ['Toothbrush_other'] 0-th fold train set: (250449, 150) -> 545 positive 249904 negative sample
	category: 29 ['Toothbrush_other'] test set: (109185, 150) -> 229 positive 108956 negative sample
[[108855    101]
 [   229      0]]
	category: 29 ['Toothbrush_other'] n_est: 100 max_depth: -1 -> acc_vl: 0.996978 auc_vl: 0.500606
	category: 29 ['Toothbrush_other'] prec: 0.000000 (0 out of 101) recall: 0.000000 (0 out of 229)
category: 30 ['Doorhandle'] 
	category: 30 ['Doorhandle'] 0-th fold train set: (250449, 150) -> 923 positive 249526 negative sample
	category: 30 ['Doorhandle'] test set: (109185, 150) -> 855 positive 108330 negative sample
[[107900    430]
 [   759     96]]
	category: 30 ['Doorhandle'] n_est: 100 m

[[108554    233]
 [   337     61]]
	category: 4 ['Frog'] n_est: 100 max_depth: -1 -> acc_vl: 0.994780 auc_vl: 0.923586
	category: 4 ['Frog'] prec: 0.207483 (61 out of 294) recall: 0.153266 (61 out of 398)
category: 5 ['Bear'] 
	category: 5 ['Bear'] 0-th fold train set: (250449, 150) -> 1471 positive 248978 negative sample
	category: 5 ['Bear'] test set: (109185, 150) -> 763 positive 108422 negative sample
[[108019    403]
 [   683     80]]
	category: 5 ['Bear'] n_est: 100 max_depth: -1 -> acc_vl: 0.990054 auc_vl: 0.966526
	category: 5 ['Bear'] prec: 0.165631 (80 out of 483) recall: 0.104849 (80 out of 763)
category: 6 ['Seal'] 
	category: 6 ['Seal'] 0-th fold train set: (250449, 150) -> 1848 positive 248601 negative sample
	category: 6 ['Seal'] test set: (109185, 150) -> 844 positive 108341 negative sample
[[107538    803]
 [   540    304]]
	category: 6 ['Seal'] n_est: 100 max_depth: -1 -> acc_vl: 0.987700 auc_vl: 0.961902
	category: 6 ['Seal'] prec: 0.274616 (304 out of 1107) recall: 

[[108792    103]
 [   260     30]]
	category: 23 ['Glass_lifting'] n_est: 100 max_depth: -1 -> acc_vl: 0.996675 auc_vl: 0.564421
	category: 23 ['Glass_lifting'] prec: 0.225564 (30 out of 133) recall: 0.103448 (30 out of 290)
category: 24 ['Glass_to_mouth'] 
	category: 24 ['Glass_to_mouth'] 0-th fold train set: (250449, 150) -> 61 positive 250388 negative sample
	category: 24 ['Glass_to_mouth'] test set: (109185, 150) -> 92 positive 109093 negative sample
[[109088      5]
 [    92      0]]
	category: 24 ['Glass_to_mouth'] n_est: 100 max_depth: -1 -> acc_vl: 0.999112 auc_vl: 0.500046
	category: 24 ['Glass_to_mouth'] prec: 0.000000 (0 out of 5) recall: 0.000000 (0 out of 92)
category: 25 ['Drinking'] 
	category: 25 ['Drinking'] 0-th fold train set: (250449, 150) -> 711 positive 249738 negative sample
	category: 25 ['Drinking'] test set: (109185, 150) -> 3072 positive 106113 negative sample
[[105646    467]
 [  3030     42]]
	category: 25 ['Drinking'] n_est: 100 max_depth: -1 -> acc_vl: 0.

[[100279   8546]
 [   360      0]]
	category: 42 ['Snack_eat'] n_est: 100 max_depth: -1 -> acc_vl: 0.918432 auc_vl: 0.529871
	category: 42 ['Snack_eat'] prec: 0.000000 (0 out of 8546) recall: 0.000000 (0 out of 360)
category: 43 ['Lame_fox'] 
	category: 43 ['Lame_fox'] 0-th fold train set: (250449, 150) -> 550 positive 249899 negative sample
	category: 43 ['Lame_fox'] test set: (109185, 150) -> 316 positive 108869 negative sample
[[108389    480]
 [   296     20]]
	category: 43 ['Lame_fox'] n_est: 100 max_depth: -1 -> acc_vl: 0.992893 auc_vl: 0.530862
	category: 43 ['Lame_fox'] prec: 0.040000 (20 out of 500) recall: 0.063291 (20 out of 316)
category: 44 ['Cupboard'] 
category: 45 ['Face_wash'] 
summary: acc 0.967264 auc 0.711383 (43)
Time elapsed (hh:mm:ss.ms) 2:28:27.768054
Time end (hh:mm:ss.ms) 2021-01-03 22:01:16.840724
